## Este es el primer notebook para el preprocesamiento de los data sets
### Para este momento, tenemos que unir el dataset de: Internet Firewall Data, con The UNSW-NB15 Dataset

# TODO re formatear el notebook, dado que ya solo se busca limpiar 3 datasets por separado 

In [1]:
#Importamos las librerias 

import pandas as pd

In [95]:
#Cargamos el data set
baseDf = pd.read_csv('DataSets/log2.csv')
#El basedf es el formato al que queremos llevar todos los datos

## Log2.csv tiene 12 columnas, las cuales son: 
1. Source Port
2. Destination Port
3. NAT Source Port
4. NAT Destination Port
5. Action
6. Bytes (suma de bytes sent y bytes received)
7. Bytes Sent
8. Bytes Received
9. Packets
10. Elapsed Time (sec)
11. pkts_sent
12. pkts_received

Donde action es nuestra columna target

Primero, verifiquemos que el dataset sea usable

In [96]:
baseDf['Action'].value_counts()

Action
allow         37640
deny          14987
drop          12851
reset-both       54
Name: count, dtype: int64

In [97]:
#Verificar que no tenga valores nulos
assert(baseDf.isnull().sum().sum() == 0)
#Verificar los types de las columnas
baseDf.dtypes

Source Port              int64
Destination Port         int64
NAT Source Port          int64
NAT Destination Port     int64
Action                  object
Bytes                    int64
Bytes Sent               int64
Bytes Received           int64
Packets                  int64
Elapsed Time (sec)       int64
pkts_sent                int64
pkts_received            int64
dtype: object

Aqui tenemos un pequeño inconveniente, el Action es un objecto, y necesitamos que sea un entero, por lo que vamos a cambiar los valores de la columna a 0 y 4, dado que tenemos 4 clases, las cuales son:


In [98]:
baseDf['Action'] = baseDf['Action'].map({'allow': 0, 'deny': 1, 'drop': 2, 'reset-both': 3})
baseDf.head()

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
0,57222,53,54587,53,0,177,94,83,2,30,1,1
1,56258,3389,56258,3389,0,4768,1600,3168,19,17,10,9
2,6881,50321,43265,50321,0,238,118,120,2,1199,1,1
3,50553,3389,50553,3389,0,3327,1438,1889,15,17,8,7
4,50002,443,45848,443,0,25358,6778,18580,31,16,13,18


In [99]:
baseDf.dtypes

Source Port             int64
Destination Port        int64
NAT Source Port         int64
NAT Destination Port    int64
Action                  int64
Bytes                   int64
Bytes Sent              int64
Bytes Received          int64
Packets                 int64
Elapsed Time (sec)      int64
pkts_sent               int64
pkts_received           int64
dtype: object

In [100]:
baseDf.__len__()

65532

In [101]:
#mostramos la correlacion entre las columnas y el target
baseDf.corrwith(baseDf['Action']).abs().nlargest(11)

Action                  1.000000
NAT Source Port         0.686893
NAT Destination Port    0.214765
Elapsed Time (sec)      0.170684
Destination Port        0.095110
Source Port             0.038343
Bytes Received          0.023794
pkts_received           0.021679
Packets                 0.015562
Bytes                   0.013546
pkts_sent               0.009842
dtype: float64

In [102]:
baseDf['NAT Source Port'].value_counts()

NAT Source Port
0        28432
48817       83
58638       51
50116       15
7986         5
         ...  
2063         1
33661        1
36797        1
14122        1
13485        1
Name: count, Length: 29152, dtype: int64

Aquí tenemos algo interesante, podemos ver que NAT Source Port tiene una alta correlacion con Action, pero al observar, podemos ver que la gran mayoria de los valores son 0, y el dataset no explica que significa esto. Por lo que vamos a quitar esta columna

In [103]:
filteredDf=baseDf[baseDf['Action'] == 1]
filteredDf[['Action','NAT Source Port']]

,Action,NAT Source Port
143,1,0
144,1,0
156,1,0
158,1,0
159,1,0
...,...,...
65518,1,0
65521,1,0
65522,1,0
65523,1,0


In [104]:
filteredDf['NAT Source Port'].value_counts()

NAT Source Port
0        14975
5575         2
53193        1
49630        1
59373        1
24378        1
62347        1
59312        1
51051        1
55898        1
36930        1
13747        1
Name: count, dtype: int64

Por esto, vamos a quitar NAT Source Port, y NAT Destination Port

In [105]:
baseDf = baseDf.drop(columns=['NAT Source Port','NAT Destination Port'])


In [106]:
baseDf.corrwith(baseDf['Action']).abs().nlargest(11)

Action                1.000000
Elapsed Time (sec)    0.170684
Destination Port      0.095110
Source Port           0.038343
Bytes Received        0.023794
pkts_received         0.021679
Packets               0.015562
Bytes                 0.013546
pkts_sent             0.009842
Bytes Sent            0.004570
dtype: float64

## Ahora, vamos a cargar el segundo dataset, el cual es el UNSW-NB15
Este está dividido en 5 csv. Pero, en este caso usaremos los UNSW_NB15_training-set.csv y UNSW_NB15_testing-set.csv

Las columnas y sus descripciones se puede encontrar en el NUSW-NB15_features.csv, dado que son demasiadas, no seran escritas aquí.

In [107]:
unswDf = pd.read_csv('DataSets/UNSW_NB15_training-set.csv')
print("El tamaño del dataset es: ",unswDf.__len__())
#El skiprows es para saltar la primera fila, la cual es la descripcion de las columnas, dado que lo vamos a pegar al primer df
unswTestDf = pd.read_csv('DataSets/UNSW_NB15_testing-set.csv') 
unswDf = pd.concat([unswDf,unswTestDf],ignore_index=True)
#Verificamos que se pegaran bien
print("El tamaño del dataset es: ",unswDf.__len__())

El tamaño del dataset es:  175341
El tamaño del dataset es:  257673


Ahora que tenemos el dataset de unsw, queremos reducir sus columnas para que tengan las mismas que el primer dataset, por esto, vamos a quitar las siguientes columnas:


In [108]:
unswDf.dtypes

id                     int64
dur                  float64
proto                 object
service               object
state                 object
spkts                  int64
dpkts                  int64
sbytes                 int64
dbytes                 int64
rate                 float64
sttl                   int64
dttl                   int64
sload                float64
dload                float64
sloss                  int64
dloss                  int64
sinpkt               float64
dinpkt               float64
sjit                 float64
djit                 float64
swin                   int64
stcpb                  int64
dtcpb                  int64
dwin                   int64
tcprtt               float64
synack               float64
ackdat               float64
smean                  int64
dmean                  int64
trans_depth            int64
response_body_len      int64
ct_srv_src             int64
ct_state_ttl           int64
ct_dst_ltm             int64
ct_src_dport_l

In [109]:
#vamos a usar label encoding para las columnas que son categoricas
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
unswDf['proto'] = le.fit_transform(unswDf['proto'])
unswDf['service'] = le.fit_transform(unswDf['service'])
unswDf['state'] = le.fit_transform(unswDf['state'])
unswDf['attack_cat'] = le.fit_transform(unswDf['attack_cat'])

In [110]:
unswDf.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,113,0,4,6,4,258,172,74.087490,...,1,1,0,0,0,1,1,0,6,0
1,2,0.649902,113,0,4,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,6,0
2,3,1.623129,113,0,4,8,16,364,13186,14.170161,...,1,3,0,0,0,2,6,0,6,0
3,4,1.681642,113,3,4,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,6,0
4,5,0.449454,113,0,4,10,6,534,268,33.373826,...,1,40,0,0,0,2,39,0,6,0


In [111]:
#las 10 columnas con mayor correlacion con el target
unswDf.corrwith(unswDf['label']).abs().nlargest(12)

label               1.000000
sttl                0.624082
attack_cat          0.602160
id                  0.490584
ct_state_ttl        0.476559
state               0.462972
ct_dst_sport_ltm    0.371672
swin                0.364877
dload               0.352169
dwin                0.339166
rate                0.335883
ct_src_dport_ltm    0.318518
dtype: float64

In [112]:
unswDf.dtypes

id                     int64
dur                  float64
proto                  int32
service                int32
state                  int32
spkts                  int64
dpkts                  int64
sbytes                 int64
dbytes                 int64
rate                 float64
sttl                   int64
dttl                   int64
sload                float64
dload                float64
sloss                  int64
dloss                  int64
sinpkt               float64
dinpkt               float64
sjit                 float64
djit                 float64
swin                   int64
stcpb                  int64
dtcpb                  int64
dwin                   int64
tcprtt               float64
synack               float64
ackdat               float64
smean                  int64
dmean                  int64
trans_depth            int64
response_body_len      int64
ct_srv_src             int64
ct_state_ttl           int64
ct_dst_ltm             int64
ct_src_dport_l

## ¿Ahora, qué haremos?
Observando los dos datasets, podemos ver que los dos son muy diferentes y las correlaciones varian mucho. Dado que no comparten casi columnas, no nos parece una buena idea unirlos, por ende, vamos a hacer dos modelos, uno para cada dataset, y veremos cual tiene mejor desempeño.

#### Limpieza de datos de baseDf

In [113]:
#Primero, vamos a quitar las columnas que no tienen correlacion con el target
baseDf.corrwith(baseDf['Action']).abs().nlargest(11)

Action                1.000000
Elapsed Time (sec)    0.170684
Destination Port      0.095110
Source Port           0.038343
Bytes Received        0.023794
pkts_received         0.021679
Packets               0.015562
Bytes                 0.013546
pkts_sent             0.009842
Bytes Sent            0.004570
dtype: float64

In [114]:
#Como podemos ver, las columnas de packets, pkts_received, bytes_received, no tienen correlacion con el target, por lo que las vamos a quitar
baseDf = baseDf.drop(columns=['pkts_received','Packets','Bytes Received'])
#Ahora vamos a guardar este dataset
baseDf.to_csv('DataSets/Internet_firewall_data.csv',index=False)

#### Limpieza de datos de unswDf

In [115]:
#Primero, vamos a quitar las columnas que no tienen correlacion con el target
unswDf.corrwith(unswDf['label']).abs().nlargest(45)
#Podemos ver, que en este datasets tenemos una correlaciones mucho más altas que en el anterior
#Eso sí, se tiene que tener en cuenta que attack_cat es una columna que nos indica si es un ataque o no
#Además, esta ID, que no tiene correlation con el target al ser una variable que solo identifica a la fila

label                1.000000
sttl                 0.624082
attack_cat           0.602160
id                   0.490584
ct_state_ttl         0.476559
state                0.462972
ct_dst_sport_ltm     0.371672
swin                 0.364877
dload                0.352169
dwin                 0.339166
rate                 0.335883
ct_src_dport_ltm     0.318518
ct_dst_src_ltm       0.299609
dmean                0.295173
stcpb                0.266585
dtcpb                0.263543
ct_src_ltm           0.252498
ct_srv_dst           0.247812
ct_srv_src           0.246596
ct_dst_ltm           0.240776
sload                0.165249
is_sm_ips_ports      0.160126
sinpkt               0.155454
dpkts                0.097394
dloss                0.075961
service              0.073552
dbytes               0.060403
djit                 0.048819
synack               0.043250
spkts                0.043040
dinpkt               0.030136
dur                  0.029096
smean                0.028372
tcprtt    

In [116]:
#Vamos a quitar las columnas que tengan una correlacion menor a 0.1
columansToDrop = unswDf.columns[unswDf.corrwith(unswDf['label']).abs() < 0.1]
unswDf = unswDf.drop(columns=columansToDrop)

In [117]:
unswDf.corrwith(unswDf['label']).abs().nlargest(45)

label               1.000000
sttl                0.624082
attack_cat          0.602160
id                  0.490584
ct_state_ttl        0.476559
state               0.462972
ct_dst_sport_ltm    0.371672
swin                0.364877
dload               0.352169
dwin                0.339166
rate                0.335883
ct_src_dport_ltm    0.318518
ct_dst_src_ltm      0.299609
dmean               0.295173
stcpb               0.266585
dtcpb               0.263543
ct_src_ltm          0.252498
ct_srv_dst          0.247812
ct_srv_src          0.246596
ct_dst_ltm          0.240776
sload               0.165249
is_sm_ips_ports     0.160126
sinpkt              0.155454
dtype: float64

In [118]:
#Ahora vamos a guardar este dataset
unswDf.to_csv('DataSets/UNSW.csv',index=False)

## Ahora, vamos a limpiar el dataset de KDDCup99

In [15]:
#Primero, vamos a ver las columnas que tiene
cols="""duration, protocol_type,
service,
flag,
src_bytes,
dst_bytes,
land,
wrong_fragment,
urgent,
hot,
num_failed_logins,
logged_in,
num_compromised,
root_shell,
su_attempted,
num_root,
num_file_creations,
num_shells,
num_access_files,
num_outbound_cmds,
is_host_login,
is_guest_login,
count,
srv_count,
serror_rate,
srv_serror_rate,
rerror_rate,
srv_rerror_rate,
same_srv_rate,
diff_srv_rate,
srv_diff_host_rate,
dst_host_count,
dst_host_srv_count,
dst_host_same_srv_rate,
dst_host_diff_srv_rate,
dst_host_same_src_port_rate,
dst_host_srv_diff_host_rate,
dst_host_serror_rate,
dst_host_srv_serror_rate,
dst_host_rerror_rate,
dst_host_srv_rerror_rate,
attack"""

#Ahora, pasemos esto a un arreglo para darle a nuestro dataframe
cols = cols.replace("\n", "")
cols = cols.split(",")
print(cols)


['duration', ' protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'attack']


In [16]:
#Primero, vamos a cargar el dataset, dado que es un flat file, no necesitamos especificar el separador
kddDf = pd.read_csv('DataSets/kddcup.data_10_percent',names=cols)


In [17]:
kddDf.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [23]:
#tenemos algo interesante, y es que attack tiene varios labels, con el archivo training_attack_types.txt podemos ver que hay 22 labels, pero solo 4 ataques, por lo que los vamos a agrupar por tipo de ataques

tipos_de_ataques = {
    'normal': 'normal',
'back': 'dos',
'buffer_overflow': 'u2r',
'ftp_write': 'r2l',
'guess_passwd': 'r2l',
'imap': 'r2l',
'ipsweep': 'probe',
'land': 'dos',
'loadmodule': 'u2r',
'multihop': 'r2l',
'neptune': 'dos',
'nmap': 'probe',
'perl': 'u2r',
'phf': 'r2l',
'pod': 'dos',
'portsweep': 'probe',
'rootkit': 'u2r',
'satan': 'probe',
'smurf': 'dos',
'spy': 'r2l',
'teardrop': 'dos',
'warezclient': 'r2l',
'warezmaster': 'r2l',
}

kddDf['Attack Type'] = kddDf.attack.apply(lambda r:tipos_de_ataques[r[:-1]])

kddDf.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack,Attack Type
0,0,tcp,http,SF,181,5450,0,0,0,0,...,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.,normal
1,0,tcp,http,SF,239,486,0,0,0,0,...,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.,normal
2,0,tcp,http,SF,235,1337,0,0,0,0,...,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.,normal
3,0,tcp,http,SF,219,1337,0,0,0,0,...,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.,normal
4,0,tcp,http,SF,217,2032,0,0,0,0,...,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.,normal


In [25]:
#Ahora, vamos a añadir una columna binaria para saber si es un ataque o no
kddDf['Is attack'] = kddDf['Attack Type'].map({'normal': 0, 'dos': 1, 'u2r': 1, 'r2l': 1, 'probe': 1})
kddDf.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack,Attack Type,Is attack
0,0,tcp,http,SF,181,5450,0,0,0,0,...,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.,normal,0
1,0,tcp,http,SF,239,486,0,0,0,0,...,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.,normal,0
2,0,tcp,http,SF,235,1337,0,0,0,0,...,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.,normal,0
3,0,tcp,http,SF,219,1337,0,0,0,0,...,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.,normal,0
4,0,tcp,http,SF,217,2032,0,0,0,0,...,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.,normal,0


In [26]:
kddDf.isnull().sum()

duration                       0
 protocol_type                 0
service                        0
flag                           0
src_bytes                      0
dst_bytes                      0
land                           0
wrong_fragment                 0
urgent                         0
hot                            0
num_failed_logins              0
logged_in                      0
num_compromised                0
root_shell                     0
su_attempted                   0
num_root                       0
num_file_creations             0
num_shells                     0
num_access_files               0
num_outbound_cmds              0
is_host_login                  0
is_guest_login                 0
count                          0
srv_count                      0
serror_rate                    0
srv_serror_rate                0
rerror_rate                    0
srv_rerror_rate                0
same_srv_rate                  0
diff_srv_rate                  0
srv_diff_h

In [ ]:
#Por ahora, no vamos a modificar las variables categoricas, dado que antes queremos hacer un analisis exploratorio